# Plan your trip with Kayak

Kayak is a travel search engine that helps users plan their next trip at the best price.

They did discover that 70% of their users who are planning a trip would like to have more information about the destination they are going to.

Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays; the application should do recommandations about the best destinations based on weather informations and hotels in the area.

# Content
- 1- Scope of the project
- 2- Weather information collection
    - 2-1 Latitudes/ longitudes collection from nominatim
    - 2-2 Weather information collection from openweathermap
    - 2-3 Creation of a dataframe
    - 2-4 Best destinations based on weather forecast
- 3- Hotels information collection
    - 3-1 Scarp of booking.com
    - 3-2 Creation of a dataframe with hotel information
    - 3-3 Top 20 hotels
- 4- Merge of all informations in a single dataframe
- 5- Creation of a data lake using S3
- 6- ETL and final maps
- Conclusion

In [1]:
import pandas as pd

import requests
import json

import datetime
from datetime import datetime

import plotly.express as px

import boto3
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text

# 1- Scope of the project

- The project will be held on the top-35 cities to visit in France, put in the below list of cities:

In [2]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [3]:
len(cities)

35

- Note here that some of the items are not cities but areas, for example "Gorges du Verdon"

# 2- Weather information collection

## 2-1 Latitudes/ longitudes collection from nominatim

- Using the API from Nominatim https://nominatim.org/ to get the gps coordinates of all the cities

In [4]:
# launching a request based on the expected structure expected of this API, detailed in API documentation
#Using of 'q=' instead of 'city=' in the API request because some sites are not cities (eg 'gorges du Verdon" is not a city but an area)

json_all_cities = []

for city in cities:
   response_cities = requests.get('https://nominatim.openstreetmap.org/search?q={}&country=fr&format=json&limit=1'.format(city))
   
   if response_cities.status_code == 200:
        json_all_cities.append(response_cities.json())
   else: 
        print("Request to {} failed".format(city))

- creating a list of latitudes based on json response

In [5]:
latitudes = []

for city in range(len(cities)):
   latitude_cities = [json_all_cities[city][0]["lat"]]
   latitudes.append(latitude_cities)
len(latitudes)

35

35 items are gathered here => ok

- creating a list of longitudes based on json response

In [6]:
longitudes = []

for city in range(len(cities)):
   longitude_cities = [json_all_cities[city][0]["lon"]]
   longitudes.append(longitude_cities)
len(longitudes)

35

35 items collected on logitudes as well => ok

## 2-2 Weather information collection from openweathermap

- Using the API from openweathermap https://openweathermap.org/appid, the weather forecast on 5 days will be collected (more than 5 days is no longer a free version of the API)
- The data collected from the API gives information every 3 hours
- Latitudes and longitudes lists created above are used as an input of the request to gather information on the cities

In [7]:
json_weather = []

for l in range(len(cities)):
        response_weather = requests.get('https://api.openweathermap.org/data/2.5/forecast?lat={}&lon={}&units=metric&appid=688f62ea0cc03358aa2135e88d1001c6'.format(latitudes[l][0], longitudes[l][0]))
           
        if response_weather.status_code == 200:
                json_weather.append(response_weather.json())
        else: 
                print("Request to {} failed".format(l))

- Only the relevant information will be picked from the json:

In [8]:
weather_details = []

for c in range(len(cities)):
    for forecast in range(len(json_weather[c]["list"])):
        weather_by_day = [json_weather[c]["city"]["name"],
            json_weather[c]["list"][forecast]["dt"],
            json_weather[c]["list"][forecast]["weather"][0]["description"],
            json_weather[c]["list"][forecast]["main"]["feels_like"],
            json_weather[c]["city"]["coord"]["lat"], 
            json_weather[c]["city"]["coord"]["lon"]]
        weather_details.append(weather_by_day)
# looking at the first 5 items of the list created
weather_details[:5]

[['Huisnes-sur-Mer', 1675328400, 'overcast clouds', 6.33, 48.636, -1.5115],
 ['Huisnes-sur-Mer', 1675339200, 'overcast clouds', 6.74, 48.636, -1.5115],
 ['Huisnes-sur-Mer', 1675350000, 'overcast clouds', 7.79, 48.636, -1.5115],
 ['Huisnes-sur-Mer', 1675360800, 'broken clouds', 7.2, 48.636, -1.5115],
 ['Huisnes-sur-Mer', 1675371600, 'overcast clouds', 7.52, 48.636, -1.5115]]

## 2-3 Creation of a dataframe

- Creating a dataframe based on the list created above

In [9]:
df = pd.DataFrame(data = weather_details, columns=["city_API","time", "weather_desc", "feels_like", "lat", "lon"])

- Looking at basic statistics

In [10]:
print("Number of rows, columns : {}".format(df.shape))
print()

print("Display of dataset: ")
display(df.head())
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows, columns : (1400, 6)

Display of dataset: 


,city_API,time,weather_desc,feels_like,lat,lon
0,Huisnes-sur-Mer,1675328400,overcast clouds,6.33,48.636,-1.5115
1,Huisnes-sur-Mer,1675339200,overcast clouds,6.74,48.636,-1.5115
2,Huisnes-sur-Mer,1675350000,overcast clouds,7.79,48.636,-1.5115
3,Huisnes-sur-Mer,1675360800,broken clouds,7.20,48.636,-1.5115
4,Huisnes-sur-Mer,1675371600,overcast clouds,7.52,48.636,-1.5115



Basics statistics: 


,city_API,time,weather_desc,feels_like,lat,lon
count,1400,1.400000e+03,1400,1400.000000,1400.000000,1400.000000
unique,35,NaN,7,NaN,NaN,NaN
top,Huisnes-sur-Mer,NaN,clear sky,NaN,NaN,NaN
freq,40,NaN,540,NaN,NaN,NaN
mean,NaN,1.675539e+09,NaN,3.818971,45.840894,3.396149
std,NaN,1.247132e+05,NaN,4.566896,2.553588,2.912522
min,NaN,1.675328e+09,NaN,-13.660000,42.525100,-2.026000
25%,NaN,1.675434e+09,NaN,1.150000,43.494500,1.355000
50%,NaN,1.675539e+09,NaN,4.285000,45.187600,4.360100
75%,NaN,1.675644e+09,NaN,6.772500,48.584600,5.735800



Percentage of missing values: 


city_API        0.0
time            0.0
weather_desc    0.0
feels_like      0.0
lat             0.0
lon             0.0
dtype: float64

- Time column needs to be converted before being used
- The name of cities gathered from the API may differ from the ones initially expected; The cities will thus have to be renamed
- The weather forecast information needs to be aggregated as day level since the API gives update on the weather forecast every 3 hours 
- 'Feels like' temperature will be rounded
- No missing data to treat

- Converting column 'time' to datetime to be able to use it afterwards

In [11]:
df['day'] = df['time'].apply(lambda x : datetime.utcfromtimestamp(x).strftime('%d-%m-%Y'))
# dropping initial col 'time' in a final df
df_def = df.drop(columns = ['time'])

- Checking names of cities gathered from the API

In [12]:
df_def['city_API'].unique()

array(['Huisnes-sur-Mer', 'St-Malo', 'Bayeux', 'Le Havre', 'Rouen',
       'Palais-Royal', 'Amiens', 'Lille', 'Strasbourg', 'Saint-Hippolyte',
       'Colmar', 'Eguisheim', 'Besançon', 'Larrey', 'Annecy', 'Grenoble',
       'Vieux Lyon', 'Bauduen', 'Bormes-les-Mimosas', 'Cassis',
       'Marseille', 'Aix-en-Provence', 'Avignon', 'Uzès', 'Nîmes',
       'Aigues-Mortes', 'Saintes-Maries-de-la-Mer', 'Collioure',
       'Carcassonne', 'Larbont', 'Toulouse', 'Montauban', 'Biarritz',
       'Le Limpou', 'La Rochelle'], dtype=object)

- Renaming cities that are no longer matching with initial names given

In [13]:
df_def["city_label"] = ["Mont Saint Michel" if cit =='Huisnes-sur-Mer'
                    else  "St Malo" if cit =='St-Malo'
                    else  "Bayeux" if cit == 'Bayeux'
                    else  "Le Havre" if cit == 'Le Havre'
                    else  "Rouen" if cit == 'Rouen'
                    else  "Paris" if cit == 'Palais-Royal'
                    else  "Amiens" if cit == 'Amiens'
                    else  "Lille" if cit == 'Lille'
                    else  "Strasbourg" if cit == 'Arrondissement de Strasbourg'
                    else  "Strasbourg" if cit == 'Strasbourg'
                    else  "Chateau du Haut Koenigsbourg" if cit == 'Saint-Hippolyte'
                    else  "Colmar" if cit == 'Colmar'
                    else  "Eguisheim" if cit == 'Eguisheim'
                    else  "Besancon" if cit == 'Besançon'
                    else  "Dijon" if cit == 'Larrey'
                    else  "Annecy" if cit ==  'Annecy'
                    else  "Grenoble" if cit == 'Grenoble'
                    else  "Lyon" if cit ==  'Vieux Lyon'
                    else  "Gorges du Verdon" if cit ==  'Bauduen'
                    else  "Bormes les Mimosas" if cit ==  'Bormes-les-Mimosas'
                    else  "Cassis" if cit == 'Cassis'
                    else  "Marseille" if cit ==  'Marseille'
                    else  "Aix en Provence" if cit == 'Aix-en-Provence'
                    else  "Avignon" if cit == 'Avignon'
                    else  "Uzes" if cit == 'Uzès'
                    else  "Nimes" if cit ==  'Nîmes'
                    else  "Aigues Mortes" if cit =='Aigues-Mortes'
                    else  "Saintes Maries de la mer" if cit == 'Saintes-Maries-de-la-Mer'
                    else  "Collioure" if cit == 'Collioure'
                    else  "Carcassonne" if cit ==  'Carcassonne'
                    else  "Ariege" if cit ==   'Larbont'
                    else  "Toulouse" if cit == 'Toulouse'
                    else  "Montauban" if cit == 'Montauban'
                    else  "Biarritz" if cit ==  'Biarritz'
                    else  "Biarritz" if cit ==  'La Négresse'
                    else "Bayonne" if cit == 'Bayonne'
                    else  "Bayonne" if cit ==   'Le Limpou'
                    else  "La Rochelle" if cit == 'La Rochelle'
                    else None
                    for cit in df['city_API']]

- Checking if any city has label 'nan'

In [14]:
# checking if any city has a label 'nan'
df_def['city_label'].isnull().sum()

0

No city has label 'nan', so we are good to go

- Checking the number of cities listed in our dataframe

In [15]:
df_def['city_label'].nunique()

35

35 cities as expected

- Aggregation of the weather info by day

In [16]:
agg_dict = {'weather_desc': 'max', 'feels_like': 'mean', 'lat': 'mean', 'lon': 'mean' }  
df_weather = df_def.groupby(['city_label','day']).agg(agg_dict).reset_index()

- Rounding feels_like temperature

In [17]:
df_weather["feels_like"] = round(df_weather["feels_like"],2)

In [18]:
df_weather.head()

,city_label,day,weather_desc,feels_like,lat,lon
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913
1,Aigues Mortes,03-02-2023,scattered clouds,6.94,43.5658,4.1913
2,Aigues Mortes,04-02-2023,clear sky,8.64,43.5658,4.1913
3,Aigues Mortes,05-02-2023,clear sky,8.13,43.5658,4.1913
4,Aigues Mortes,06-02-2023,scattered clouds,3.33,43.5658,4.1913


## 2-4 Best destinations based on weather forecast

To define the best destinations to go, weather description and feels like temperature will be used.

A score will be attributed to each weather description (best being no clouds, no rain).

The feels like temperature will be a score as well.

These 2 scores will be added together, and summed on the 5 days. The 5 best total scores in the end will define the 5 best cities to go.

- Checking the possible values of weather description

In [19]:
df_weather['weather_desc'].unique()

array(['clear sky', 'scattered clouds', 'few clouds', 'overcast clouds',
       'light rain', 'broken clouds', 'light snow'], dtype=object)

- Attributing a score to each of the weather description. 

Let's consider that rain is definitly not expected during the trip; thus whatever rain weather will bring a negative score.

On the controry a clear sky will bring a high score. Intermediate is clouds, and are thus scored in the middle.

Some of the scores have been left empty in case new weather descriptions pop from the list above if this code is run in the future.

In [20]:
df_weather['weather_desc_score'] = df_weather['weather_desc'].apply(lambda x: 10 if x == 'xx' # empty for further use
                                                                            else 9 if x == 'xx' # empty for further use
                                                                            else 8 if x == 'clear sky'
                                                                            else 7 if x == 'few clouds' 
                                                                            else 6 if x == 'scattered clouds'
                                                                            else 5 if x == 'broken clouds'
                                                                            else 4 if x == 'overcast clouds'
                                                                            else 3 if x == 'light snow'
                                                                            else 2 if x == 'snow'
                                                                            else -10 if x == 'light rain'
                                                                            else -20 if x == 'moderate rain' 
                                                                            else -30 if x == 'heavy intensity rain' 
                                                                            else  'xx' # empty for further use
                                                                            ) 
df_weather.head()

,city_label,day,weather_desc,feels_like,lat,lon,weather_desc_score
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8
1,Aigues Mortes,03-02-2023,scattered clouds,6.94,43.5658,4.1913,6
2,Aigues Mortes,04-02-2023,clear sky,8.64,43.5658,4.1913,8
3,Aigues Mortes,05-02-2023,clear sky,8.13,43.5658,4.1913,8
4,Aigues Mortes,06-02-2023,scattered clouds,3.33,43.5658,4.1913,6


- Adding-up feels_like temperature + weather_desc_score:

In [21]:
df_weather['best_cities'] = df_weather['feels_like']+df_weather['weather_desc_score']
df_weather.head()

,city_label,day,weather_desc,feels_like,lat,lon,weather_desc_score,best_cities
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11
1,Aigues Mortes,03-02-2023,scattered clouds,6.94,43.5658,4.1913,6,12.94
2,Aigues Mortes,04-02-2023,clear sky,8.64,43.5658,4.1913,8,16.64
3,Aigues Mortes,05-02-2023,clear sky,8.13,43.5658,4.1913,8,16.13
4,Aigues Mortes,06-02-2023,scattered clouds,3.33,43.5658,4.1913,6,9.33


- Group by city to find the score over the 5 days

In [22]:
cities_dict = {"lat":"mean", "lon":"mean", "best_cities":"sum"}
best_cities = df_weather.groupby('city_label').agg(cities_dict).reset_index()
best_cities_sorted = best_cities.sort_values('best_cities', ascending = False, ignore_index = True)
best_cities_sorted.head()

,city_label,lat,lon,best_cities
0,Cassis,43.2140,5.5396,87.74
1,Collioure,42.5251,3.0832,83.40
2,Marseille,43.2962,5.3700,82.29
3,Saintes Maries de la mer,43.4523,4.4287,79.11
4,Aigues Mortes,43.5658,4.1913,74.56


Gathering here latitude and longitude that will be used in the final graph.

- Creating the list of the top 5 cities (will be used later on):

In [23]:
top_5_cities = []
for i in range(5): # 5 best cities
    top_cit = best_cities_sorted['city_label'][i]
    top_5_cities.append(top_cit)
top_5_cities

['Cassis',
 'Collioure',
 'Marseille',
 'Saintes Maries de la mer',
 'Aigues Mortes']

- Creating a column 'size': this will allow that the top 5 destinations appear bigger on the final map

In [24]:
df_weather['size']= df_weather['city_label'].apply(lambda x : 100 if x in top_5_cities else 10)
df_weather.head()

,city_label,day,weather_desc,feels_like,lat,lon,weather_desc_score,best_cities,size
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100
1,Aigues Mortes,03-02-2023,scattered clouds,6.94,43.5658,4.1913,6,12.94,100
2,Aigues Mortes,04-02-2023,clear sky,8.64,43.5658,4.1913,8,16.64,100
3,Aigues Mortes,05-02-2023,clear sky,8.13,43.5658,4.1913,8,16.13,100
4,Aigues Mortes,06-02-2023,scattered clouds,3.33,43.5658,4.1913,6,9.33,100


# 3- Hotels information collection

## 3-1 Scrap of booking.com

This information is collected through scraping https://www.booking.com/

The code is stored in the file Kayak_booking.py

To launch this file, type "python Kayak_booking.py" on the terminal from the folder where the file is stored.

# 3-2 Creation of a dataframe with hotel information                       

- Pulling the information stored in hotel_details.json (collected with scrapy)

In [25]:
with open('src/hotel_details.json') as source:
    data = json.load(source)

In [26]:
# looking at the first item
data[0]

{'city': 'Le Mont Saint Michel: 13 properties found',
 'name': 'Apparthôtel Mont Saint Michel - Résidence Fleurdumont',
 'rating': '8.0',
 'desc': 'Résidence Fleurdumont offers pet-friendly accommodation in Beauvoir. Mont Saint Michel Abbey is 4.4 km away. ',
 'url': 'https://www.booking.com/hotel/fr/residence-fleurdumont.en-gb.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaE2IAQGYAQm4AQrIAQXYAQHoAQH4AQOIAgGoAgO4Av3H6p4GwAIB0gIkZWE3YTVhZDAtNjZiMy00Zjc0LWJlOTEtZjQ4ZGRmYTkwYTBl2AIG4AIB&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=25&hapos=25&sr_order=popularity&srpvid=adbc7c3f935c025e&srepoch=1675273216&from=searchresults#hotelTmpl',
 'gps': '48.59648217,-1.50314692'}

- Creating a dataframe

In [27]:
hotel = pd.DataFrame(data = data, columns=["city","name", "rating", "desc", "url", "gps"])

- Looking at basic stats

In [28]:
print("Number of rows, columns : {}".format(hotel.shape))
print()

print("Display of dataset: ")
display(hotel.head())
print()

print("Basics statistics: ")
data_desc = hotel.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*hotel.isnull().sum()/hotel.shape[0])

Number of rows, columns : (875, 6)

Display of dataset: 


,city,name,rating,desc,url,gps
0,Le Mont Saint Michel: 13 properties found,Apparthôtel Mont Saint Michel - Résidence Fleu...,8.0,Résidence Fleurdumont offers pet-friendly acco...,https://www.booking.com/hotel/fr/residence-fle...,"48.59648217,-1.50314692"
1,Saint Malo: 492 properties found,Le haut solidor,None,"Offering free WiFi and sea views, Le haut soli...",https://www.booking.com/hotel/fr/le-haut-solid...,"48.63543333,-2.02211480"
2,Le Mont Saint Michel: 13 properties found,Gites Bellevue,9.3,Located just 2 km from the famous Mont Saint-M...,https://www.booking.com/hotel/fr/gites-bellevu...,"48.60788007,-1.51722372"
3,Le Mont Saint Michel: 13 properties found,Au Mont Chez Nous Jaccuzi et Sauna,9.2,"Located in Pontorson, 8.6 km from Mont Saint M...",https://www.booking.com/hotel/fr/26-route-de-l...,"48.61345530,-1.48587230"
4,Strasbourg: 457 properties found,Appartement Petite France avec terrasse,9.3,"Set in the centre of Strasbourg, 700 metres fr...",https://www.booking.com/hotel/fr/appartement-p...,"48.57902876,7.74253450"



Basics statistics: 


,city,name,rating,desc,url,gps
count,875,875,714,733,875,875
unique,35,872,47,733,875,863
top,Le Mont Saint Michel: 13 properties found,Oustaou di Ercole,9.0,Résidence Fleurdumont offers pet-friendly acco...,https://www.booking.com/hotel/fr/residence-fle...,"44.01122460,4.41992260"
freq,25,2,61,1,1,2



Percentage of missing values: 


city       0.000000
name       0.000000
rating    18.400000
desc      16.228571
url        0.000000
gps        0.000000
dtype: float64

- Name of cities has to be cleaned: only the name has to be kept and probably we need to check if the names are the same as in the original list
- GPS information will have to be split in latitudes/ longitudes
- Description field may be shortened so that the final map remains readable
- URLs can be reduced and cut after "?"
- The rating does not look numeric. As it needs to be sorted to find the best hotels it will have to be transformed to numeric
- Some information are missing on rating and description; this will be fine for our project

-  Cleaning of city name, URL and splitting gps info into latitude and longitude, shorten description

In [29]:
# Splitting columns city, gps and url
hotel[['city_label','to_drop']] = hotel.city.str.split(":",expand=True)
hotel[['lat','lon']] = hotel.gps.str.split(",",expand=True)
hotel[['url','url_to_drop']] = hotel.url.str.split("?",expand=True)

# Adding a character "?" that was used as a separator but is needed to find the page
hotel["url"] = hotel["url"] + "?"

# limiting description to 100 caracters
hotel["desc"] = hotel['desc'].str[:100]

# transforming lat, lon  and rating to numeric datatypes
hotel["lat"] = pd.to_numeric(hotel["lat"])
hotel["lon"] = pd.to_numeric(hotel["lon"])
hotel['rating'] = pd.to_numeric(hotel['rating'])

# rounding Lat and lon
hotel["lat"] = round(hotel["lat"],4)
hotel["lon"] = round(hotel["lon"],4)

# dropping columns that are no longer necessary
hotel.drop(columns= ["city","to_drop", "gps", "url_to_drop"], inplace = True)

- Checking names of cities gathered from booking

In [30]:
hotel["city_label"].unique()

array(['Le Mont Saint Michel', 'Saint Malo', 'Strasbourg',
       'Le Château du Haut-Koenigsbourg', 'Amiens', 'Grenoble', 'Annecy',
       'Eguisheim', 'Dijon', 'Colmar', 'Besançon', 'Lille', 'Rouen',
       'Paris', 'Bayeux', 'Le Havre', 'La Rochelle', 'Bayonne',
       'Montauban', 'Biarritz', 'Toulouse', 'Marseille',
       'Aix-en-Provence', 'Collioure', 'Uzès', 'Avignon', 'Nîmes',
       'Aigues-Mortes', 'Saintes-Maries-de-la-Mer', 'Ariège',
       'Carcassonne', 'Cassis', 'Lyon', 'Bormes-les-Mimosas',
       'Gorges du Verdon'], dtype=object)

In [31]:
len(hotel["city_label"].unique())

35

There are 35 cities as expected but the names need to be aligned with the initial list, for example 'Le Mont Saint Michel' needs to be 'Mont Saint Michel"

- Let's rename the cities

In [32]:
hotel["city_label"] = ["Mont Saint Michel" if cit =='Le Mont Saint Michel'
                    else  "St Malo" if cit =='Saint Malo'
                    else  "Bayeux" if cit == 'Bayeux'
                    else  "Le Havre" if cit == 'Le Havre'
                    else  "Rouen" if cit == 'Rouen'
                    else  "Paris" if cit == 'Paris'
                    else  "Amiens" if cit == 'Amiens'
                    else  "Lille" if cit == 'Lille'
                    else  "Strasbourg" if cit == 'Strasbourg'
                    else  "Chateau du Haut Koenigsbourg" if cit == 'Le Château du Haut-Koenigsbourg'
                    else  "Colmar" if cit == 'Colmar'
                    else  "Eguisheim" if cit == 'Eguisheim'
                    else  "Besancon" if cit == 'Besançon'
                    else  "Dijon" if cit == 'Dijon'
                    else  "Annecy" if cit ==  'Annecy'
                    else  "Grenoble" if cit == 'Grenoble'
                    else  "Lyon" if cit ==  'Lyon'
                    else  "Gorges du Verdon" if cit ==  'Gorges du Verdon'
                    else  "Bormes les Mimosas" if cit ==  'Bormes-les-Mimosas'
                    else  "Cassis" if cit == 'Cassis'
                    else  "Marseille" if cit ==  'Marseille'
                    else  "Aix en Provence" if cit == 'Aix-en-Provence'
                    else  "Avignon" if cit == 'Avignon'
                    else  "Uzes" if cit == 'Uzès'
                    else  "Nimes" if cit ==  'Nîmes'
                    else  "Aigues Mortes" if cit =='Aigues-Mortes'
                    else  "Saintes Maries de la mer" if cit == 'Saintes-Maries-de-la-Mer'
                    else  "Collioure" if cit == 'Collioure'
                    else  "Carcassonne" if cit ==  'Carcassonne'
                    else  "Ariege" if cit ==   'Ariège'
                    else  "Toulouse" if cit == 'Toulouse'
                    else  "Montauban" if cit == 'Montauban'
                    else  "Biarritz" if cit ==  'Biarritz'
                    else  "Bayonne" if cit ==   'Bayonne'
                    else  "La Rochelle" if cit == 'La Rochelle'
                    else None
                    for cit in hotel['city_label']]

- and check if any city has a label 'nan'

In [33]:
hotel['city_label'].isnull().sum()

0

In [34]:
hotel.head()

,name,rating,desc,url,city_label,lat,lon
0,Apparthôtel Mont Saint Michel - Résidence Fleu...,8.0,Résidence Fleurdumont offers pet-friendly acco...,https://www.booking.com/hotel/fr/residence-fle...,Mont Saint Michel,48.5965,-1.5031
1,Le haut solidor,NaN,"Offering free WiFi and sea views, Le haut soli...",https://www.booking.com/hotel/fr/le-haut-solid...,St Malo,48.6354,-2.0221
2,Gites Bellevue,9.3,Located just 2 km from the famous Mont Saint-M...,https://www.booking.com/hotel/fr/gites-bellevu...,Mont Saint Michel,48.6079,-1.5172
3,Au Mont Chez Nous Jaccuzi et Sauna,9.2,"Located in Pontorson, 8.6 km from Mont Saint M...",https://www.booking.com/hotel/fr/26-route-de-l...,Mont Saint Michel,48.6135,-1.4859
4,Appartement Petite France avec terrasse,9.3,"Set in the centre of Strasbourg, 700 metres fr...",https://www.booking.com/hotel/fr/appartement-p...,Strasbourg,48.5790,7.7425


All cities seem to be properly named.

## 3-3 Top 20 hotels

- Let's sort the rating of hotels

In [35]:
hotel_sorted = hotel.sort_values(['city_label', 'rating'],
              ascending = [True, False])

- Let's now flag the top 20 hotels: these will be the ones having the highest rating from booking.com, and being in the top 5 destinations defined earlier

In [36]:
top_hotel = []

for best_cit in range(len(top_5_cities)):
    df_hotel=hotel_sorted[hotel_sorted['city_label']==top_5_cities[best_cit]].reset_index(drop = True)

    for hot in range(20):
        top_hotel_by_cit = df_hotel['name'][hot]
        top_hotel.append(top_hotel_by_cit)
# looking at the 5 first items
top_hotel[:5]

['Le Corton - 45m2 - Parking - Jardin - Wifi',
 "L’Aiguade, appartement d'exception à Cassis",
 'LE SEPT charmant studio aux portes des calanques',
 'Magnifique Appartement à Cassis,Parking,2 pas du Port',
 'Astoria Villa maison d hôtes Appartement vue mer avec piscine']

5 best cities x 20 hotels = 100 items are expected in this list

In [37]:
len(top_hotel)

100

- Flagging the top hotels in a new column that will be used later on with the final maps

In [38]:
hotel['top_hotel'] = hotel['name'].apply(lambda x :1 if x in top_hotel else 0)
hotel.head()

,name,rating,desc,url,city_label,lat,lon,top_hotel
0,Apparthôtel Mont Saint Michel - Résidence Fleu...,8.0,Résidence Fleurdumont offers pet-friendly acco...,https://www.booking.com/hotel/fr/residence-fle...,Mont Saint Michel,48.5965,-1.5031,0
1,Le haut solidor,NaN,"Offering free WiFi and sea views, Le haut soli...",https://www.booking.com/hotel/fr/le-haut-solid...,St Malo,48.6354,-2.0221,0
2,Gites Bellevue,9.3,Located just 2 km from the famous Mont Saint-M...,https://www.booking.com/hotel/fr/gites-bellevu...,Mont Saint Michel,48.6079,-1.5172,0
3,Au Mont Chez Nous Jaccuzi et Sauna,9.2,"Located in Pontorson, 8.6 km from Mont Saint M...",https://www.booking.com/hotel/fr/26-route-de-l...,Mont Saint Michel,48.6135,-1.4859,0
4,Appartement Petite France avec terrasse,9.3,"Set in the centre of Strasbourg, 700 metres fr...",https://www.booking.com/hotel/fr/appartement-p...,Strasbourg,48.5790,7.7425,0


# 4- Merge of all informations in a single dataframe

In [39]:
df_load = pd.merge(df_weather, hotel, on = "city_label", how = "outer")

- Basic stats for check

In [40]:
print("Number of rows, columns : {}".format(df_load.shape))
print()

print("Display of dataset: ")
display(df_load.head())
print()

print("Basics statistics: ")
data_desc = df_load.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df_load.isnull().sum()/df_load.shape[0])

Number of rows, columns : (5250, 16)

Display of dataset: 


,city_label,day,weather_desc,feels_like,lat_x,lon_x,weather_desc_score,best_cities,size,name,rating,desc,url,lat_y,lon_y,top_hotel
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Maison T3 jardin proche remparts Aigues Mortes,7.0,Located 25 km from Parc des Expositions de Mon...,https://www.booking.com/hotel/fr/maison-t3-ave...,43.5672,4.2050,0
1,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Marina à Aigues Mortes - GARD,9.2,"Boasting lake views, Marina à Aigues Mortes - ...",https://www.booking.com/hotel/fr/marina-a-aigu...,43.5747,4.1885,1
2,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,La maison sur la place,8.5,"Featuring a garden and a terrace, La maison su...",https://www.booking.com/hotel/fr/la-maison-sur...,43.5659,4.1930,1
3,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Résidence Odalys Fleur de Sel,8.3,None,https://www.booking.com/hotel/fr/residence-oda...,43.5741,4.1792,1
4,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Awesome home in Aigues-Mortes with 3 Bedrooms ...,9.5,Situated 24 km from Parc des Expositions de Mo...,https://www.booking.com/hotel/fr/three-bedroom...,43.5739,4.1869,1



Basics statistics: 


,city_label,day,weather_desc,feels_like,lat_x,lon_x,weather_desc_score,best_cities,size,name,rating,desc,url,lat_y,lon_y,top_hotel
count,5250,5250,5250,5250.000000,5250.000000,5250.000000,5250.000000,5250.000000,5250.000000,5250,4284.000000,4398,5250,5250.000000,5250.000000,5250.000000
unique,35,6,7,NaN,NaN,NaN,NaN,NaN,NaN,872,NaN,733,875,NaN,NaN,NaN
top,Aigues Mortes,02-02-2023,scattered clouds,NaN,NaN,NaN,NaN,NaN,NaN,Le Cosy,NaN,Located 25 km from Parc des Expositions de Mon...,https://www.booking.com/hotel/fr/maison-t3-ave...,NaN,NaN,NaN
freq,150,875,2075,NaN,NaN,NaN,NaN,NaN,NaN,12,NaN,6,6,NaN,NaN,NaN
mean,NaN,NaN,NaN,3.136619,45.840894,3.396149,5.971429,9.108048,22.857143,NaN,8.572129,NaN,NaN,45.835518,3.403128,0.114286
std,NaN,NaN,NaN,4.497407,2.552919,2.911759,1.830902,4.820181,31.496439,NaN,1.062437,NaN,NaN,2.557578,2.903286,0.318188
min,NaN,NaN,NaN,-13.280000,42.525100,-2.026000,-10.000000,-7.280000,10.000000,NaN,1.000000,NaN,NaN,42.520900,-2.027900,0.000000
25%,NaN,NaN,NaN,0.380000,43.494500,1.355000,4.000000,6.180000,10.000000,NaN,8.100000,NaN,NaN,43.495200,1.356000,0.000000
50%,NaN,NaN,NaN,4.370000,45.187600,4.360100,6.000000,10.120000,10.000000,NaN,8.800000,NaN,NaN,45.187300,4.358600,0.000000
75%,NaN,NaN,NaN,6.340000,48.584600,5.735800,7.000000,12.550000,10.000000,NaN,9.200000,NaN,NaN,48.577500,5.711800,0.000000



Percentage of missing values: 


city_label             0.000000
day                    0.000000
weather_desc           0.000000
feels_like             0.000000
lat_x                  0.000000
lon_x                  0.000000
weather_desc_score     0.000000
best_cities            0.000000
size                   0.000000
name                   0.000000
rating                18.400000
desc                  16.228571
url                    0.000000
lat_y                  0.000000
lon_y                  0.000000
top_hotel              0.000000
dtype: float64

All looks ok

# 5- Creation of a data lake using S3

- Creating a csv file from the dataframe created above

In [41]:
df_load.to_csv('df_load_to_s3.csv', index=False)

- Session on AWS

**credentials have been removed**

In [42]:
session = boto3.Session(aws_access_key_id=xxx, 
                        aws_secret_access_key=xxx)


In [43]:
# creation of variable S3 that connects the session to the S3 ressource
s3 = session.resource("s3")

# creation of variable called bucket that will connect to a bucket that is being created now
bucket = s3.create_bucket(Bucket="kayakprojectfeed") # unique name

- Loading to S3

In [44]:
s3.Bucket('kayakprojectfeed').upload_file('df_load_to_s3.csv','df_load.csv')

- Downloading from S3

In [45]:
s3.Bucket('kayakprojectfeed').download_file('df_load.csv','df_load_from_s3.csv')

- checking the file downloaded from S3

In [46]:
download = pd.read_csv("df_load_from_s3.csv")
download.head()

,city_label,day,weather_desc,feels_like,lat_x,lon_x,weather_desc_score,best_cities,size,name,rating,desc,url,lat_y,lon_y,top_hotel
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Maison T3 jardin proche remparts Aigues Mortes,7.0,Located 25 km from Parc des Expositions de Mon...,https://www.booking.com/hotel/fr/maison-t3-ave...,43.5672,4.2050,0
1,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Marina à Aigues Mortes - GARD,9.2,"Boasting lake views, Marina à Aigues Mortes - ...",https://www.booking.com/hotel/fr/marina-a-aigu...,43.5747,4.1885,1
2,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,La maison sur la place,8.5,"Featuring a garden and a terrace, La maison su...",https://www.booking.com/hotel/fr/la-maison-sur...,43.5659,4.1930,1
3,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Résidence Odalys Fleur de Sel,8.3,NaN,https://www.booking.com/hotel/fr/residence-oda...,43.5741,4.1792,1
4,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Awesome home in Aigues-Mortes with 3 Bedrooms ...,9.5,Situated 24 km from Parc des Expositions de Mo...,https://www.booking.com/hotel/fr/three-bedroom...,43.5739,4.1869,1


In [47]:
print (f"Shape of initial file: {df_load.shape}")
print (f"Shape of file downloaded from S3: {download.shape}")

Shape of initial file: (5250, 16)
Shape of file downloaded from S3: (5250, 16)


Both files have the same shape

# 6- ETL and final maps

In [48]:
download.head()

,city_label,day,weather_desc,feels_like,lat_x,lon_x,weather_desc_score,best_cities,size,name,rating,desc,url,lat_y,lon_y,top_hotel
0,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Maison T3 jardin proche remparts Aigues Mortes,7.0,Located 25 km from Parc des Expositions de Mon...,https://www.booking.com/hotel/fr/maison-t3-ave...,43.5672,4.2050,0
1,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Marina à Aigues Mortes - GARD,9.2,"Boasting lake views, Marina à Aigues Mortes - ...",https://www.booking.com/hotel/fr/marina-a-aigu...,43.5747,4.1885,1
2,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,La maison sur la place,8.5,"Featuring a garden and a terrace, La maison su...",https://www.booking.com/hotel/fr/la-maison-sur...,43.5659,4.1930,1
3,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Résidence Odalys Fleur de Sel,8.3,NaN,https://www.booking.com/hotel/fr/residence-oda...,43.5741,4.1792,1
4,Aigues Mortes,02-02-2023,clear sky,6.11,43.5658,4.1913,8,14.11,100,Awesome home in Aigues-Mortes with 3 Bedrooms ...,9.5,Situated 24 km from Parc des Expositions de Mo...,https://www.booking.com/hotel/fr/three-bedroom...,43.5739,4.1869,1


- Load to SQL database

The below elements are commented because experiencing access issues; As a workaround a simulation of SQL database will be done

In [49]:
#engine = create_engine('sqlite:///:memory:', echo=True)
#DBHOST = "xxx"
#DBUSER = "xxx"
#DBPASS = "xxx"
#DBNAME = "xxx"
#PORT = "xxx"
# DBNAME = "postgres" --> If you are using PostgreSQL

#engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)
#engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}", echo=True)

#Session = sessionmaker(bind=engine)
#Session = Session()

In [50]:
engine = create_engine("sqlite:///:memory:", echo=True) #simulates database

- Pushing the dataframe to our SQL database

In [51]:
download.to_sql(
    "kayak_load",
    engine
)

2023-02-02 09:57:45,770 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("kayak_load")
2023-02-02 09:57:45,772 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-02 09:57:45,774 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("kayak_load")
2023-02-02 09:57:45,777 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-02 09:57:45,780 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-02 09:57:45,782 INFO sqlalchemy.engine.Engine 
CREATE TABLE kayak_load (
	"index" BIGINT, 
	city_label TEXT, 
	day TEXT, 
	weather_desc TEXT, 
	feels_like FLOAT, 
	lat_x FLOAT, 
	lon_x FLOAT, 
	weather_desc_score BIGINT, 
	best_cities FLOAT, 
	size BIGINT, 
	name TEXT, 
	rating FLOAT, 
	"desc" TEXT, 
	url TEXT, 
	lat_y FLOAT, 
	lon_y FLOAT, 
	top_hotel BIGINT
)


2023-02-02 09:57:45,784 INFO sqlalchemy.engine.Engine [no key 0.00137s] ()
2023-02-02 09:57:45,788 INFO sqlalchemy.engine.Engine CREATE INDEX ix_kayak_load_index ON kayak_load ("index")
2023-02-02 09:57:45,789 INFO sqlalchemy.engine.Engine

5250

- Let's query our SQL database and create a map of the top cities:

In [52]:
# fields to be requested
stmt = text("SELECT lat_x, lon_x, feels_like, day, size, city_label FROM kayak_load")

# pulling 
df_cities_from_sql = pd.read_sql(
    stmt,
    engine
)

df_cities_from_sql.head()

2023-02-02 09:57:46,136 INFO sqlalchemy.engine.Engine SELECT lat_x, lon_x, feels_like, day, size, city_label FROM kayak_load
2023-02-02 09:57:46,137 INFO sqlalchemy.engine.Engine [generated in 0.00179s] ()


,lat_x,lon_x,feels_like,day,size,city_label
0,43.5658,4.1913,6.11,02-02-2023,100,Aigues Mortes
1,43.5658,4.1913,6.11,02-02-2023,100,Aigues Mortes
2,43.5658,4.1913,6.11,02-02-2023,100,Aigues Mortes
3,43.5658,4.1913,6.11,02-02-2023,100,Aigues Mortes
4,43.5658,4.1913,6.11,02-02-2023,100,Aigues Mortes


In [53]:
# map of all the cities, where top 5 are bigger
fig = px.scatter_mapbox(df_cities_from_sql, lat="lat_x", lon="lon_x", 
                        color="feels_like", 
                        color_continuous_scale='Bluered',
                        zoom=4.5, 
                        mapbox_style="carto-positron", 
                        animation_frame = 'day' ,
                        size= 'size', 
                        title = "35 best cities to visit - Top 5 cities to go", 
                        hover_name = df_cities_from_sql["city_label"],
                        height = 800, width = 800)
fig.show()

- Let's query our SQL database and create a map of the top hotels:

In [54]:
# fields to be requested
stmt = text("SELECT lat_y, lon_y, rating, name, url, top_hotel, city_label, desc, day FROM kayak_load where top_hotel =1")

df_hotels_from_sql = pd.read_sql(
    stmt,
    engine
)

df_hotels_from_sql.head()

2023-02-02 09:57:48,930 INFO sqlalchemy.engine.Engine SELECT lat_y, lon_y, rating, name, url, top_hotel, city_label, desc, day FROM kayak_load where top_hotel =1
2023-02-02 09:57:48,932 INFO sqlalchemy.engine.Engine [generated in 0.00193s] ()


,lat_y,lon_y,rating,name,url,top_hotel,city_label,desc,day
0,43.5747,4.1885,9.2,Marina à Aigues Mortes - GARD,https://www.booking.com/hotel/fr/marina-a-aigu...,1,Aigues Mortes,"Boasting lake views, Marina à Aigues Mortes - ...",02-02-2023
1,43.5659,4.1930,8.5,La maison sur la place,https://www.booking.com/hotel/fr/la-maison-sur...,1,Aigues Mortes,"Featuring a garden and a terrace, La maison su...",02-02-2023
2,43.5741,4.1792,8.3,Résidence Odalys Fleur de Sel,https://www.booking.com/hotel/fr/residence-oda...,1,Aigues Mortes,None,02-02-2023
3,43.5739,4.1869,9.5,Awesome home in Aigues-Mortes with 3 Bedrooms ...,https://www.booking.com/hotel/fr/three-bedroom...,1,Aigues Mortes,Situated 24 km from Parc des Expositions de Mo...,02-02-2023
4,43.5589,4.2181,9.7,AIGUES MARINES,https://www.booking.com/hotel/fr/aigues-marine...,1,Aigues Mortes,"Boasting a seasonal outdoor pool, AIGUES MARIN...",02-02-2023


In [55]:
# map of all the hotels in the top cities

for city in range(5):
    fig = px.scatter_mapbox(df_hotels_from_sql, lat="lat_y", lon="lon_y", 
                            color="rating", 
                            color_continuous_scale='matter',
                            center = {"lat": best_cities_sorted.iloc[city,1], "lon": best_cities_sorted.iloc[city,2]},
                            zoom=13, 
                            mapbox_style="carto-positron",  
                            title = "20 best hotels to go in the top 5 cities", 
                            hover_name = "name", 
                            text = df_hotels_from_sql["url"].astype(str)  + "<br>" + "desc= " +df_hotels_from_sql["desc"].astype(str),
                            height = 800, width = 800)
    fig.add_annotation(x=0, y=1,text=best_cities_sorted.iloc[city,0], showarrow=False, font_size=20)
    fig.show()

# Conclusion

2 sets of maps have been delivered:
- A map showing the top 5 cities to go out of the 35 best cities to visit in France
- A map of each top city (1 for each top city) showing the top 20 hotels with their details: Hotel name, URL, Coordinates, score, description.

To do so, the data has been collected through APIs (nomintim and openweathermap) and directly on the web by scraping booking.com.

The data has been stored on an S3, stored on an SQL database, and pulled from there to build the final maps.